# Import Packages & Read Data

In [3]:
import pandas as pd
import json
import os
import ast 
import re
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
from sklearn import preprocessing


import numpy as np
from tqdm import tqdm
import chardet
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px

from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
import textblob
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import math
from nltk import bigrams,trigrams,ngrams
from collections import Counter
from sklearn.metrics import confusion_matrix
from nltk import everygrams, word_tokenize
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_confusion_matrix

business = pd.read_json('./business.json',lines = True)
CA_business = business[business['state'] == 'CA']
CA_business = CA_business.reset_index()

# Deal with business data

### Extract object like information

In [4]:
for i in range(len(CA_business)):
    cur_attr = CA_business['attributes'][i]
    if bool(cur_attr) == False:
        continue
    for key in cur_attr.keys():
        CA_business.loc[i,key] = cur_attr[key]


def extract_info (df, feature):
    for i in range(len(df)):
        try:
            cur_attr = df[feature][i]
            cur_attr = ast.literal_eval(cur_attr)
            for key in cur_attr.keys():
                df.loc[i,key] = cur_attr[key]
        except: 
            continue

            
# def extract_info_hours (df, feature):
#     for i in range(len(df)):
#         try:
#             cur_attr = df[feature][i]
#             #cur_attr = ast.literal_eval(cur_attr)
#             for key in cur_attr.keys():
#                 df.loc[i,key] = cur_attr[key]
#         except: 
#             continue
            
def extract_object_info (df, feature):
    for i in range(len(df)):
        try:
            cur_attr = df[feature][i]
            #cur_attr = ast.literal_eval(cur_attr)
            for key in cur_attr.keys():
                df.loc[i,key] = cur_attr[key]
        except: 
            continue           
    
            
extract_info(CA_business,'BusinessParking')    
extract_info(CA_business,'GoodForMeal')     
extract_info(CA_business,'Ambience')     
extract_object_info(CA_business,'hours')     
extract_object_info(CA_business,'Music')     
extract_object_info(CA_business,'BestNights')     
        
CA_business = CA_business.drop(columns=['attributes','BusinessParking','GoodForMeal','Ambience'])
CA_business = CA_business.rename({'stars':'average_stars'},axis = 1)

### Based on word in category and name, we extracted East Asian restaurant

In [5]:
#Leave non-NA categories and clean it up, then find those contain certain words
temp_business = CA_business[CA_business['categories'].notna()].reset_index(drop = True)
temp_business['category_cleaned'] = temp_business['categories'].str.split()
temp_business['category_cleaned'] = temp_business['category_cleaned'].apply(lambda x : [re.sub(r'[.,"\'-?:!;]', '', i) for i in x])
temp_business['category_cleaned'] = temp_business['category_cleaned'].apply(lambda x : [re.sub(r'\<a href', ' ', i) for i in x])
temp_business['category_cleaned'] = temp_business['category_cleaned'].apply(lambda x : [re.sub(r'&amp;', '', i) for i in x])
temp_business['category_cleaned'] = temp_business['category_cleaned'].apply(lambda x : [re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', i) for i in x])
temp_business['category_cleaned'] = temp_business['category_cleaned'].apply(lambda x : [re.sub(r'<br />', ' ', i) for i in x])
temp_business['category_cleaned'] = temp_business['category_cleaned'].apply(lambda x : [re.sub(r'\'', ' ', i) for i in x])
temp_business['category_cleaned'] = temp_business['category_cleaned'].apply(lambda x: [item.lower() for item in x])


Asian_keyword = ['Chinese','Korean','Japanese','Asian','Sushi','Taiwan','Taiwaness','Hongkong','noodles','dumplings']
Asian_keyword = [name.lower() for name in Asian_keyword]
Asian_business_id1 = []
for i in range(len(temp_business)):
    cur_cat = temp_business.loc[i,'category_cleaned']
    try: 
        if any(item in cur_cat for item in Asian_keyword):
            Asian_business_id1.append(temp_business.loc[i,'business_id'])
    except: continue
        
        
#Leave non-NA name and clean it up, then find those contain certain words
temp_business = CA_business[CA_business['name'].notna()].reset_index(drop = True)
temp_business['name_cleaned'] = temp_business['name'].str.split()
temp_business['name_cleaned'] = temp_business['name_cleaned'].apply(lambda x : [re.sub(r'[.,"\'-?:!;]', '', i) for i in x])
temp_business['name_cleaned'] = temp_business['name_cleaned'].apply(lambda x : [re.sub(r'\<a href', ' ', i) for i in x])
temp_business['name_cleaned'] = temp_business['name_cleaned'].apply(lambda x : [re.sub(r'&amp;', '', i) for i in x])
temp_business['name_cleaned'] = temp_business['name_cleaned'].apply(lambda x : [re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', i) for i in x])
temp_business['name_cleaned'] = temp_business['name_cleaned'].apply(lambda x : [re.sub(r'<br />', ' ', i) for i in x])
temp_business['name_cleaned'] = temp_business['name_cleaned'].apply(lambda x : [re.sub(r'\'', ' ', i) for i in x])
temp_business['name_cleaned'] = temp_business['name_cleaned'].apply(lambda x: [item.lower() for item in x])


Asian_keyword = ['Chinese','Korean','Japanese','Asian','Sushi','Taiwan','Taiwaness','Hongkong','noodles','dumplings']
Asian_keyword = [name.lower() for name in Asian_keyword]
Asian_business_id2 = []
for i in range(len(temp_business)):
    cur_cat = temp_business.loc[i,'name_cleaned']
    try: 
        if any(item in cur_cat for item in Asian_keyword):
            Asian_business_id2.append(temp_business.loc[i,'business_id'])
    except: continue

In [6]:
#Extracted Asian business
Asian_business_id = set(Asian_business_id1+Asian_business_id2)
CA_Asian_food = CA_business[CA_business['business_id'].isin(Asian_business_id)].reset_index(drop = True)

### Remove records that have certain keyword that doesn't belong to what we are focusing on

In [7]:
# Remove records that has certain keywords that is related to other categories
non_restaurant_list = ['Medicine', 'Health','Meical','Acupuncture','Massage','Vietnamese','Thai','Martial Arts','Italian','Grocery','Indonesian']
non_restuarant_id = []
for i in range(len(CA_Asian_food)):
    cur_cat = CA_Asian_food.loc[i,'categories']
    try: 
        if any(item in cur_cat for item in non_restaurant_list):
            non_restuarant_id.append(CA_Asian_food.loc[i,'business_id'])
    except: continue
        
CA_Asian_food = CA_Asian_food[~CA_Asian_food['business_id'].isin(non_restuarant_id)].reset_index(drop = True)

### Turn Monday - Sunday working hour to length

In [8]:
def convert_hour(time):
    result = 0
    if len(time) == 5:
        if time[-2] == '00':
            result = int(time[:-3])
        elif int(time[-2]) > 15:
            result = int(time[:-3])+1
        else:
            result = int(time[:-3])
    elif len(time) == 4:
        if time[-1] == '0':
            result = int(time[:-2])
        elif int(time[-1]) > 15:
            result = int(time[:-2])+1
        else:
            result = int(time[:-2])
    else:
        if time[-1] == '0':
            result = int(time[:-2])
        elif int(time[-1]) > 15:
            result = int(time[:-2])+1
        else:
            result = int(time[:-2])
    return result

def calculate_total_hour(feature, new_feature):
    for i in range(len(CA_Asian_food)):
        try:
            cur_schedule =  CA_Asian_food.loc[i,feature]
            ls = cur_schedule.split('-')
            if ls[0] == ls[1]:
                CA_Asian_food.loc[i,new_feature] = 24
            else:
                if(ls[1] == '0:0'):
                    CA_Asian_food.loc[i,new_feature] = 24 - convert_hour(ls[0])
                elif(convert_hour(ls[1]) < convert_hour(ls[0])):
                    CA_Asian_food.loc[i,new_feature] = 24 + convert_hour(ls[1]) - convert_hour(ls[0])
                else:
                    work_hour = convert_hour(ls[1])-convert_hour(ls[0])
                    CA_Asian_food.loc[i,new_feature] = work_hour
        except: 
            continue
        

calculate_total_hour('Monday','Monday_working_length')
calculate_total_hour('Tuesday','Tuesday_working_length')
calculate_total_hour('Wednesday','Wednesday_working_length')
calculate_total_hour('Thursday','Thursday_working_length')
calculate_total_hour('Friday','Friday_working_length')
calculate_total_hour('Saturday','Saturday_working_length')
calculate_total_hour('Sunday','Sunday_working_length')



### Based on the above cell of code, calculate weekly working hour

In [9]:
#Calcualte weekly working hour
CA_Asian_food['Total_hour'] = CA_Asian_food[['Monday_working_length','Tuesday_working_length',
                                                                  'Wednesday_working_length','Thursday_working_length',
                                                                  'Friday_working_length','Saturday_working_length',
                                                                  'Sunday_working_length']].sum(axis = 1)

### Categorize restaurant category

In [12]:
Chinese_word_list = ['Chinese','Bubble Tea','Szechuan','Dim Sum','Hot Pot']
Japanese_word_list =['Japanese','Sushi','Sushi Bars','Ramen']
Korean_word_list = ['Korean','Soul Food']
Fusion_list = ['Asian Fusion','Fusion']

#当同一个category里的词同时出现在两个及以上的list里面时算fusion
for i in range(len(CA_Asian_food)):
    cur_category = CA_Asian_food.loc[i,'categories'].split(',')
    cur_category = [x.strip() for x in cur_category]
    fusion_count = 0
    C_count = 0
    J_count = 0
    K_count = 0
    
    if any(elem in Fusion_list for elem in cur_category):
        CA_Asian_food.loc[i,'general_category'] = 'Asian Fusion'
        continue
    else:
        if any(elem in Chinese_word_list for elem in cur_category):
            fusion_count += 1
            C_count += 1
        if any(elem in Japanese_word_list for elem in cur_category):
            fusion_count += 1
            J_count +=1 
        if any(elem in Korean_word_list for elem in cur_category):
            fusion_count += 1
            K_count += 1
            
    if fusion_count > 1:
        CA_Asian_food.loc[i,'general_category'] = 'Asian Fusion'
    else:
        if C_count != 0:
            CA_Asian_food.loc[i,'general_category'] = 'Chinese'
        if J_count != 0:
            CA_Asian_food.loc[i,'general_category'] = 'Japanese'
        if K_count != 0:
            CA_Asian_food.loc[i,'general_category'] = 'Korean'
            
CA_Asian_food = CA_Asian_food.drop(['categories'],axis = 1)

### Unifing feature type

In [13]:
non_alcohol_list = ["u'none'" ,"'none'",]
alcohol_list = [ "u'beer_and_wine'" , "'beer_and_wine'" ,"u'full_bar'","'full_bar'"]
for i in range(len(CA_Asian_food)):
    cur_alcohol = CA_Asian_food.loc[i,'Alcohol']
    if cur_alcohol in non_alcohol_list:
        CA_Asian_food.loc[i,'Alcohol'] = False
    elif cur_alcohol in alcohol_list:
        CA_Asian_food.loc[i,'Alcohol'] = True
    else: 
        CA_Asian_food.loc[i,'Alcohol'] = None
        
wifi_list =  ["u'free'" ,"'free'"] 
no_wifi_list = ["u'no'", "'no'"]
for i in range(len(CA_Asian_food)):
    cur_wifi = CA_Asian_food.loc[i,'WiFi']
    if cur_wifi in no_wifi_list:
        CA_Asian_food.loc[i,'WiFi'] = False
    elif cur_wifi in wifi_list:
        CA_Asian_food.loc[i,'WiFi'] = True
    else: 
        CA_Asian_food.loc[i,'WiFi'] = None
        
ok_noise =  ["u'average'", "u'quiet'","'average'" ,"'quiet'"] 
not_ok_noise = ["u'loud'","'loud'"]
for i in range(len(CA_Asian_food)):
    cur_noise = CA_Asian_food.loc[i,'NoiseLevel']
    if cur_noise in ok_noise:
        CA_Asian_food.loc[i,'acceptable_noise'] = True
    elif cur_noise in not_ok_noise:
        CA_Asian_food.loc[i,'acceptable_noise'] = False
    else: 
        CA_Asian_food.loc[i,'acceptable_noise'] = None
        
CA_Asian_food = CA_Asian_food.drop(['NoiseLevel'], axis = 1)

### Keep only opening restaurant

In [15]:
CA_Asian_food = CA_Asian_food[CA_Asian_food['is_open'] != 0].reset_index(drop = True)

### Drop feature that will not be used

In [16]:
CA_Asian_food = CA_Asian_food.drop(['Music','BestNights','Smoking','BYOBCorkage','BYOB','GoodForDancing','AgesAllowed'],axis = 1)

# Combine Reivew dataset with business

In [17]:
review = pd.read_csv('./review.csv')
review = review.rename({'stars':'comment_star'},axis = 1)
frames = [CA_Asian_food, review]
CA_Asian_business_review = pd.merge(review, CA_Asian_food,left_on='business_id',right_on = 'business_id', how = 'right' ).reset_index(drop=True)

### Categorize sentiment

In [18]:
#Categorize each sentiment
def categorize_sentiment(row):
    if row['comment_star'] > 3:
        return 'Positive'
    elif row['comment_star'] == 3:
        return 'Neutral'
    elif row['comment_star'] < 3:
        return 'Negative'
CA_Asian_business_review['sentiment'] = CA_Asian_business_review.apply(lambda row : categorize_sentiment(row), axis = 1)



### Clean comment text

In [19]:
#Append more words to stop word list based on word cloud
stop = stopwords.words('english')
stop.append('Santa Barbara')
stop.append('santa barbara')
stop.append('Santa')
stop.append('Barbara')
stop.append('santa')
stop.append('barbara')
stop.append('chinese restaurant')
stop.append('korean bbq')
stop.append('Korean bbq')
stop.append('place')
stop.append('u')
stop.append('iv')
stop.append('id')
stop.append('i am')

#Remove stopwords
lemm = WordNetLemmatizer()
#stop = set(stopwords.words('english'))

CA_Asian_business_review['text_word_list'] = CA_Asian_business_review['text'].str.split()
CA_Asian_business_review['text_no_stopwords'] = CA_Asian_business_review['text_word_list'].apply(lambda x : [re.sub(r'[.,"\'-?:!;]', '', i) for i in x])
CA_Asian_business_review['text_no_stopwords'] = CA_Asian_business_review['text_no_stopwords'].apply(lambda x : [re.sub(r'\<a href', ' ', i) for i in x])
CA_Asian_business_review['text_no_stopwords'] = CA_Asian_business_review['text_no_stopwords'].apply(lambda x : [re.sub(r'&amp;', '', i) for i in x])
CA_Asian_business_review['text_no_stopwords'] = CA_Asian_business_review['text_no_stopwords'].apply(lambda x : [re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', i) for i in x])
CA_Asian_business_review['text_no_stopwords'] = CA_Asian_business_review['text_no_stopwords'].apply(lambda x : [re.sub(r'<br />', ' ', i) for i in x])
CA_Asian_business_review['text_no_stopwords'] = CA_Asian_business_review['text_no_stopwords'].apply(lambda x : [re.sub(r'\'', ' ', i) for i in x])
CA_Asian_business_review['text_no_stopwords'] = CA_Asian_business_review['text_no_stopwords'].apply(lambda x: [item.lower() for item in x])
CA_Asian_business_review['text_no_stopwords'] = CA_Asian_business_review['text_no_stopwords'].apply(lambda x : [i for i in x if i not in stop])
CA_Asian_business_review['text_cleaned'] = [' '.join(map(str,l)) for l in CA_Asian_business_review['text_no_stopwords']]

def lemmatize_text(text):
    return [lemm.lemmatize(word) for word in nltk.WordPunctTokenizer().tokenize(text)]
CA_Asian_business_review['text_cleaned'] = CA_Asian_business_review.text_cleaned.apply(lemmatize_text)

### Save to local file

In [20]:
CA_Asian_business_review.to_csv('CA_Asian_business_review.csv',index = False)

# Extracted dataset for modeling

### Extracted features that we are interested in

In [24]:
modeling_feature_list = ['review_id', 'user_id', 'business_id', 'comment_star', 'date',
      'name', 'city', 'average_stars', 'review_count',
      'BusinessAcceptsCreditCards',
       'BikeParking',  'RestaurantsTakeOut',
       'RestaurantsReservations',
       'RestaurantsTableService', 'GoodForKids', 'WheelchairAccessible',
       'RestaurantsDelivery', 'HasTV', 
       'Alcohol', 'DogsAllowed', 'RestaurantsGoodForGroups',
       'WiFi', 'BusinessAcceptsBitcoin', 'CoatCheck', 'DriveThru',
       'RestaurantsCounterService',
       'garage', 'street',
       'lot', 'valet', 'dessert', 'latenight', 'lunch', 'dinner', 'brunch',
       'breakfast', 'hipster', 'divey', 'intimate',
       'trendy', 'upscale', 'classy', 'casual', 'Total_hour', 'acceptable_noise','general_category']
ambiance_list = ['hipster', 'divey', 'intimate','trendy', 'upscale', 'classy', 'casual']
modeling_df = CA_Asian_business_review[modeling_feature_list]

### Convert ambient feature to true, false and None (hipster,upscale, divey, etc)

In [25]:
for i in range(len(modeling_df)):
    cur_hip =  modeling_df.loc[i,'hipster']
    cur_trend = modeling_df.loc[i,'trendy']
    if  (cur_hip == True) | (cur_trend == True):
        modeling_df.loc[i,'hipster_trendy'] = True
    elif ((cur_hip != True) & (cur_hip != False)) & ((cur_trend != True) & (cur_trend != False)):
        modeling_df.loc[i,'hipster_trendy'] = None
    else:
         modeling_df.loc[i,'hipster_trendy'] = False
            
    cur_up = modeling_df.loc[i,'upscale']
    cur_clas = modeling_df.loc[i,'classy']
    if  (cur_up == True) | (cur_clas == True):
        modeling_df.loc[i,'upscale_classy'] = True
    elif ((cur_up != True) & (cur_up != False)) & ((cur_clas != True) & (cur_clas != False)):
        modeling_df.loc[i,'upscale_classy'] = None
    else:
         modeling_df.loc[i,'upscale_classy'] = False
            
    cur_dive = modeling_df.loc[i,'divey']
    if cur_dive == True:
        modeling_df.loc[i,'divey_env'] = True
    elif cur_dive == False:
        modeling_df.loc[i,'divey_env'] = False
    else:
        modeling_df.loc[i,'divey_env'] = None
    
    cur_inti = modeling_df.loc[i,'intimate']
    if cur_inti == True:
        modeling_df.loc[i,'intimate_env'] = True
    elif cur_inti == False:
        modeling_df.loc[i,'intimate_env'] = False
    else:
        modeling_df.loc[i,'intimate_env'] = None
        
    cur_casual = modeling_df.loc[i,'casual']
    if cur_casual == True:
        modeling_df.loc[i,'casual_env'] = True
    elif cur_casual == False:
        modeling_df.loc[i,'casual_env'] = False
    else:
        modeling_df.loc[i,'casual_env'] = None

modeling_df = modeling_df.drop(ambiance_list,axis = 1)

### Convert other feautres to true, false and None

In [26]:
other_feature_list = ['BusinessAcceptsCreditCards', 'BikeParking', 'RestaurantsTakeOut',
        'RestaurantsReservations', 'RestaurantsTableService',
       'GoodForKids', 'WheelchairAccessible', 'RestaurantsDelivery', 'HasTV',
       'DogsAllowed', 'RestaurantsGoodForGroups',
       'BusinessAcceptsBitcoin', 'CoatCheck', 'DriveThru',
       'RestaurantsCounterService', 'garage', 'street', 'lot', 'valet',
       'dessert', 'latenight', 'lunch', 'dinner', 'brunch', 'breakfast']



for feature in other_feature_list:
   for i in range(len(modeling_df)):
    cur_cell = modeling_df.loc[i,feature]
    if (cur_cell == 'True') | (cur_cell == True):
        modeling_df.loc[i,feature] = True
    elif (cur_cell == 'False') | (cur_cell == False):
        modeling_df.loc[i,feature] = False
    else:
         modeling_df.loc[i,feature] = None
            
modeling_df = modeling_df.reset_index(drop = True)

### Save to local file

In [27]:
modeling_df.to_csv('modeling_df.csv',index = False)